# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']
J. Li  ->  J. Li  |  ['J. Li']
S. Li  ->  S. Li  |  ['S. Li']


Arxiv has 53 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.10845


extracting tarball to tmp_2309.10845...

 done.


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']


Found 61 bibliographic references in tmp_2309.10845/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2309.11016


extracting tarball to tmp_2309.11016...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.11090


extracting tarball to tmp_2309.11090...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.10845-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.10845) | **Water absorption in the transmission spectrum of the water-world  candidate GJ9827d**  |
|| P.-A. Roy, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>T. Mikal-Evans</mark> |
|*Appeared on*| *2023-09-21*|
|*Comments*| *Published in ApJL, 11 pages, 6 figures*|
|**Abstract**| Recent work on the characterization of small exoplanets has allowed us to accumulate growing evidence that the sub-Neptunes with radii greater than $\sim2.5\,R_\oplus$ often host H$_2$/He-dominated atmospheres both from measurements of their low bulk densities and direct detections of their low mean-molecular-mass atmospheres. However, the smaller sub-Neptunes in the 1.5-2.2 R$_\oplus$ size regime are much less understood, and often have bulk densities that can be explained either by the H$_2$/He-rich scenario, or by a volatile-dominated composition known as the "water world" scenario. Here, we report the detection of water vapor in the transmission spectrum of the $1.96\pm0.08$ R$_\oplus$ sub-Neptune GJ9827d obtained with the Hubble Space Telescope. We observed 11 HST/WFC3 transits of GJ9827d and find an absorption feature at 1.4$\mu$m in its transit spectrum, which is best explained (at 3.39$\sigma$) by the presence of water in GJ9827d's atmosphere. We further show that this feature cannot be caused by unnoculted star spots during the transits by combining an analysis of the K2 photometry and transit light-source effect retrievals. We reveal that the water absorption feature can be similarly well explained by a small amount of water vapor in a cloudy H$_2$/He atmosphere, or by a water vapor envelope on GJ9827d. Given that recent studies have inferred an important mass-loss rate ($>0.5\,$M$_\oplus$/Gyr) for GJ9827d making it unlikely to retain a H-dominated envelope, our findings highlight GJ9827d as a promising water world candidate that could host a volatile-dominated atmosphere. This water detection also makes GJ9827d the smallest exoplanet with an atmospheric molecular detection to date. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.11016-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.11016) | **A Gigantic Mid-Infrared Outburst in an Embedded Class-I Young Stellar  Object J064722.95+031644.6**  |
|| T. Wang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-09-21*|
|*Comments*| *15 pages, 7 figures, accepted to ApJ*|
|**Abstract**| We report the serendipitous discovery of a giant mid-infrared (MIR) outburst from a previously unknown source near a star-forming region in the constellation Monoceros. The source gradually brightened by a factor of 5 from 2014 to 2016 before an abrupt rise by a factor of more than 100 in 2017. A total amplitude increase of >500 at 4.5 microns has since faded by a factor of about 10. Prior to the outburst, it was only detected at wavelengths longer than 1.8 microns in UKIDSS, Spitzer, and Herschel with a spectral energy distribution of a Class I Young Stellar Object (YSO). It has not been detected in recent optical surveys, suggesting that it is deeply embedded. With a minimum distance of 3.5 kpc, the source has a bolometric luminosity of at least 9 $L_\odot$ in the quiescent state and 400 $L_\odot$ at the peak of the eruption. The maximum accretion rate is estimated to be at least a few $10^{-5}$ $M_\odot$ year$^{-1}$. It shares several common properties with another eruptive event, WISE~J142238.82-611553.7: exceptionally large amplitude, featureless near-infrared spectrum with the exception of H_2 lines, intermediate eruption duration, an embedded Class I YSO, and a low radiative temperature (<600-700 K) in outburst. We interpret that the radiation from the inner accretion disk and young star is obscured and reprocessed by either an inflated outer disk or thick dusty outflow on scales > 6.5 AU during the outburst. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.11090-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.11090) | **Optimal map-making with singularities**  |
|| Z. Zhang, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-09-21*|
|*Comments*| *24 pages, 7 figures, and 2 appendices*|
|**Abstract**| In this work, we investigate the optimal map-making technique for the linear system $\bm{d}=\bm{A}\bm{x}+\bm{n}$ while carefully taking into account singularities that may come from either the covariance matrix $\bm{C} = \<\bm{n}\bm{n}^t\>$ or the main matrix $\bm{A}$. We first describe the general optimal solution, which is quite complex, and then use the modified pseudo inverse to create a near-optimal solution, which is simple, robust, and can significantly alleviate the unwanted noise amplification during map-making. The effectiveness of the nearly optimal solution is then compared to that of the naive co-adding solution and the standard pseudo inverse solution, showing noticeable improvements. Interestingly, all one needs to get the near-optimal solution with singularity is just a tiny change to the traditional optimal solution that is designed for the case without singularity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.10845.md
    + _build/html/tmp_2309.10845/./ReRun_SpectraFit_range_with_star_May17.png
    + _build/html/tmp_2309.10845/./GJ9827d_2D_posterior_H2OVsClouds_mar27.png
    + _build/html/tmp_2309.10845/./GJ9827d_AllWLCLightCurves_dev_May19.png
    + _build/html/tmp_2309.10845/./GJ9827d_RetrievalSamplesAndForwardModels_June28.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# $\large{Water absorption in the transmission spectrum of the water-world candidate GJ 9827 d}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.10845-b31b1b.svg)](https://arxiv.org/abs/2309.10845)<mark>Appeared on: 2023-09-21</mark> -  _Published in ApJL, 11 pages, 6 figures_

</div>
<div id="authors">

P.-A. Roy, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>T. Mikal-Evans</mark>

</div>
<div id="abstract">

**Abstract:** Recent work on the characterization of small exoplanets has allowed us to accumulate growing evidence that the sub-Neptunes with radii greater than $\sim2.5 R_\oplus$ often host $H_2$ /He-dominated atmospheres both from measurements of their low bulk densities and direct detections of their low mean-molecular-mass atmospheres.However, the smaller sub-Neptunes in the 1.5-2.2 R $_\oplus$ size regime are much less understood, and often have bulk densities that can be explained either by the $H_2$ /He-rich scenario, or by a volatile-dominated composition known as the "water world" scenario.Here, we report the detection of water vapor in the transmission spectrum of the $1.96\pm0.08$ R $_\oplus$ sub-Neptune GJ 9827 d obtained with the Hubble Space Telescope.We observed 11 HST/WFC3 transits of GJ 9827 d and find an absorption feature at 1.4 $\mu$ m in its transit spectrum, which is best explained (at 3.39 $\sigma$ ) by the presence of water in GJ 9827 d's atmosphere. We further show that this feature cannot be caused by unnoculted star spots during the transits by combining an analysis of the K2 photometry and transit light-source effect retrievals. We reveal that the water absorption feature can be similarly well explained by a small amount of water vapor in a cloudy $H_2$ /He atmosphere, or by a water vapor envelope on GJ 9827 d.Given that recent studies have inferred an important mass-loss rate ( $>0.5 $ M $_\oplus$ /Gyr) for GJ 9827 d making it unlikely to retain a H-dominated envelope, our findings highlight GJ 9827 d as a promising water world candidate that could host a volatile-dominated atmosphere. This water detection also makes GJ 9827 d the smallest exoplanet with an atmospheric molecular detection to date.

</div>

<div id="div_fig1">

<img src="tmp_2309.10845/./ReRun_SpectraFit_range_with_star_May17.png" alt="Fig4.1" width="50%"/><img src="tmp_2309.10845/./GJ9827d_2D_posterior_H2OVsClouds_mar27.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Water detection in the transmission spectrum of GJ 9827 d. **Left:** Transmission spectrum of GJ 9827 d (black points) shown with our model transmission spectra constraints from the nested sampling atmosphere retrieval (blue) and from the photometry-informed "transit light-source effect" retrieval (orange). The dark blue and light blue shaded regions show the 1$\sigma$ and 2$\sigma$ Bayesian credible intervals  from the atmosphere retrieval respectively. The atmospheric median transmission model is shown in blue and the best-fitting model is shown in red. The best-fitting TLS model is shown in orange along with the 1$\sigma$ and 2$\sigma$ Bayesian credible intervals in light orange. **Right:** Joint constraints on the cloud-top pressure versus the water mixing ratio derived from our Scarlet well-mixed retrieval. The colored shading describes the normalized probability density as a function of the water mixing ratio (assuming uniform vertical profiles) of the atmosphere, and of the cloud-top pressure. The black contours highlight the 1, 2 and 3$\sigma$ Bayesian credible regions. The water abundance relative to a solar composition atmosphere is shown on the top axis. The posterior probability distribution allows for multiple atmospheric scenarios ranging from $H_2$/He envelopes with small amounts of water to water-dominated envelopes. The blue points identify two representative samples of these two scenarios which are displayed in Figure \ref{fig:spectrum_forward}. (*fig:spectrum_retrieval*)

</div>
<div id="div_fig2">

<img src="tmp_2309.10845/./GJ9827d_AllWLCLightCurves_dev_May19.png" alt="Fig2" width="100%"/>

**Figure 2. -** All 10 HST/WFC3 broadband light-curve fits of the transits of GJ 9827 d. **Left:** Systematics-corrected and normalized broadband light curves for the 10 transits of GJ 9827 d (data points). Each visit is centered around the fitted transit time for that visit. The best-fitting transit model is also shown as the grey line. **Right:** Residuals of the broadband light-curve fits shown on the left. (*fig:wlc*)

</div>
<div id="div_fig3">

<img src="tmp_2309.10845/./GJ9827d_RetrievalSamplesAndForwardModels_June28.png" alt="Fig6" width="100%"/>

**Figure 6. -** HST/WFC3 transmission spectrum of GJ 9827 d (data points) along with SCARLET forward atmosphere models (colored lines). **Top:** Two samples from our well-mixed retrieval analysis (Figure \ref{fig:spectrum_retrieval}) are shown, representing the mini-Neptune scenario with a cloudy $H_2$/He atmosphere composed of $\sim$1\% water (pale blue) and a water world scenario with a water-rich atmosphere (dark blue). **Middle:** A secondary atmosphere model for a pure methane envelope is also shown (red) in order to highlight the methane absorption features. Chemically consistent models (still assuming a uniform temperature profile) are shown for a cloud-free, solar composition case (solar metallicity, solar C/O; orange) and for a cloudy case with C/O=0.1  and a 100 $\times$ solar metallicity (green). The observed spectrum is inconsistent with cloud-free low-metallicity scenarios and prefers water absorption features to methane absorption features, mainly around 1.2 and 1.65 $\mu$m. The strength of the features in the spectrum is also displayed in units of H/He scale heights (right axis). **Bottom:** The best-fit model from the retrieval analysis is shown (pale blue), along with the transmission spectrum of the same model once the water opacity is turned off (grey). The contribution of water opacity to the spectral signatures is highlighted in blue. We also present a binned version of the transmission spectrum where points are binned together by pair with the exception of the blue-most channel. (*fig:spectrum_forward*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.10845"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

276  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
